# Lab 2 - Term-Document-Matrix

Εγκατάσταση BeautifulSoup για να κάνουμε scrapping

In [1]:
pip install beautifulsoup4

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Δημιουργία `http request` και λήψη `html` σελίδας χαρακτήρων των έργων του Shakespeare: https://www.opensourceshakespeare.org/views/plays/characters/chardisplay.php  (εκτύπωση πρώτων 200 χαρακτήρων)

In [4]:
from urllib.request import urlopen

url = "https://www.opensourceshakespeare.org/views/plays/characters/chardisplay.php"
page = urlopen(url)
html = page.read().decode("utf-8")
print(html[0:200])


<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head>
<title> Shakespeare characters, sorted alphabetically :|: Open Source Shakespeare &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 


Χρήση BeautifulSoup για να πάρουμε τα δεδομένα του πίνακα που έχει ονόματα χαρακτήρων και σε ποια έργα εμφανίζονται.  
_Κάντε Inspect Elements από το browser σας για να βρείτε τον πίνακα με τα ονόματα._

In [48]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html, "html.parser")

characters = {}
plays = set()

tables = soup.find_all('table')
for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td', attrs={'class':'chartable'})
        if len(cols)==0:
            continue;
        # print(cols)
        c = cols[1].get_text()
        characters[ c ] = []
        plays_list = cols[3].find_all('p')
        for p in plays_list:
            characters[ c ].append(p.get_text().strip())
            plays.add(p.get_text().strip())
            
print('Characters: ', len(characters) )
print('Plays: ', len(plays) )


Characters:  928
Plays:  37


In [35]:
pip install more-itertools

     |████████████████████████████████| 54 kB 641 kB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [45]:
from more_itertools import take

print( take(5, characters.items() ) )

[('Aaron', ['Titus Andronicus']), ('Abbot', ['Richard II']), ('Abhorson', ['Measure for Measure']), ('Abraham', ['Romeo and Juliet']), ('Achilles', ['Troilus and Cressida'])]


Δημιουργούμε ένα binary array το οποίο αναπαριστά το Term-Document Matrix, για κάθε _χαρακτήρα_ χρειάζεται ένα bitarray με πλάτος όσο το πλήθος των έργων. 

In [49]:
pip install bitarray

     |████████████████████████████████| 229 kB 1.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [70]:
from bitarray import bitarray

plays_list = list(plays) # in order to have indexes
TDM = {}
for c in characters:
    TDM[c] = bitarray( len(plays) ) #one bit per play for each character
    TDM[c].setall(0) # initialise all bits to 0 = character does not appear in play
    for p in characters[c]:
        index = plays_list.index(p)
        # print(p, index)
        (TDM[c])[index] = 1
            
for c in TDM:
    print( '{0: <30}'.format(c[0:30]), TDM[c].to01()) 

Aaron                          0000000000000000000000000000000001000
Abbot                          0001000000000000000000000000000000000
Abhorson                       0000000000000000000000000001000000000
Abraham                        0000000100000000000000000000000000000
Achilles                       0000000000000001000000000000000000000
Adam                           0000000000000000000100000000000000000
Adrian                         0000000000000000001000000000000000000
Adriana                        0000000000001000000000000000000000000
Aedile                         0000000000000000000000000000000100000
Aegeon                         0000000000001000000000000000000000000
Aemilia                        0000000000001000000000000000000000000
Aemilius                       0000000000000000000000000000000001000
Aeneas                         0000000000000001000000000000000000000
Agamemnon                      0000000000000001000000000000000000000
Agrippa                        000

Get in which plays certain characters co-exist, eg. Brutus AND Ceasar, using bitwise operations:


In [105]:
term1 = TDM['Romeo']
term2 = TDM['Juliet']
res = (term1 & term2).to01()
res

'0000000100000000000000000000000000000'

In [106]:
foundAt = res.find('1', 0)
foundAt
while foundAt > -1 :
    print( plays_list[foundAt] )
    foundAt = res.find('1', foundAt+1)

Romeo and Juliet


:-)